In [1]:
import torch.utils.data as data

import random
import numbers
from PIL import Image, ImageMath
import os
import os.path
import numpy as np
import struct
import math

import torch
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import faiss
import time

import pcl
from PIL import Image, ImageDraw
import faiss

import timeit

%matplotlib qt5

In [2]:
class KNNBuilder_GPU:
    def __init__(self, k):
        self.k = k
        self.dimension = 3
        
        # we need only a StandardGpuResources per GPU
        self.res = faiss.StandardGpuResources()
        self.res.setTempMemoryFraction(0.1)
        self.flat_config = faiss.GpuIndexFlatConfig()
        self.flat_config.device = 0
        
    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.GpuIndexFlatL2(self.res, self.dimension, self.flat_config)  # dimension is 3
        index.add(database)
        return index
    
    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I
    
    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I
    

class KNNBuilder:
    def __init__(self, k):
        self.k = k
        self.dimension = 3

    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.IndexFlatL2(self.dimension)  # dimension is 3
        index.add(database)
        return index

    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I

    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I
    

class PCSampler:
    def __init__(self, leaf_size, minimum_pc_num):       
        self.leaf_size = leaf_size
        self.minimum_pc_num = minimum_pc_num
    
    def sample_pc(self, pc, leaf_size):
        '''
        :param pc: input numpy array of Nx3
        :return: sampled_pc of Mx3
        '''
        cloud = pcl.PointCloud(pc)
        sor = cloud.make_voxel_grid_filter()
        sor.set_leaf_size(leaf_size, leaf_size, leaf_size)
        cloud_filtered = sor.filter()
        sampled_pc = np.asarray(cloud_filtered)
        
        return sampled_pc
    
    def sample_pc_wrapper(self, pc):
        '''
        ensure that the sampled pc is more than a certain amount
        '''
        retry_counter = 0
        
        sampled_pc = self.sample_pc(pc, self.leaf_size)
        while sampled_pc.shape[0] < self.minimum_pc_num:
            retry_counter += 1
            leaf_size = self.leaf_size - 0.04*retry_counter
            if leaf_size <= 0:
                break
            sampled_pc = self.sample_pc(pc, leaf_size)
        
        return sampled_pc
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)
        
        

In [3]:
def load_train_txt(txt_path):
    f = open(txt_path, 'r')
    lines_list = f.readlines()
    
    dataset = []
    for i, line_str in enumerate(lines_list):
        # convert each line to a dict
        line_splitted_list = line_str.split('|')
        try:
            assert len(line_splitted_list) == 3
        except Exception:
            print('Invalid line.')
            print(i)
            print(line_splitted_list)
            continue
        
        file_name = line_splitted_list[0].strip()
        positive_lines = list(map(int, line_splitted_list[1].split()))
        non_negative_lines = list(map(int, line_splitted_list[2].split()))
        
#         print(file_name)
#         print(positive_lines)
#         print(non_negative_lines)

        data = {'file': file_name, 'pos_list': positive_lines, 'nonneg_list': non_negative_lines}
        dataset.append(data)
        
    f.close()
    return dataset


def load_bin(file_path, num_cols=6):
    pc_np = np.fromfile(file_path, dtype=np.float32)
    pc_np = np.reshape(pc_np, (-1, num_cols))
    
    # remove the surface normals, I will compute it myself using python-pcl
    return pc_np[:, 0:3]

In [ ]:
# get statistic and visualization of data provided by 3dfeatnet
train_folder_path = '/ssd/dataset/oxford/train'
train_txt_path = '/ssd/dataset/oxford/train_relative.txt'

test_folder_path = '/ssd/dataset/oxford/test_models'
test_txt_path = '/ssd/dataset/oxford/test_models/groundtruths.txt'  # with rotation, translation gt, for each pair

dataset = load_train_txt(train_txt_path)

In [ ]:
# test
for i, data in enumerate(dataset):
    if i<1100:
        continue
    print('---')
    
    # original pc
    pc_np = load_bin(os.path.join(train_folder_path, data['file']))
    print(pc_np.shape)
    
    ###### knn analysis
    knn_k=2
    knn = KNNBuilder(k=knn_k)

    start_t = timeit.default_timer()
    index = knn.build_nn_index(np.ascontiguousarray(pc_np[:, 0:3], dtype=np.float32))
    D, I = knn.search_nn(index, np.ascontiguousarray(pc_np[:, 0:3], dtype=np.float32), k=knn_k)
    stop_t = timeit.default_timer()
    print(stop_t - start_t)

    D = np.sqrt(np.fabs(D))
    D = D[:, knn_k-1:knn_k]
    print(D.shape)
    print('pc k=%d - mean %f, max %f, min %f' % (knn_k, np.mean(D), np.max(D), np.min(D)))
    ###### knn analysis
    
    # outlier filer
    fil = pcl.PointCloud(pc_np).make_statistical_outlier_filter()
    fil.set_mean_k(50)
    fil.set_std_dev_mul_thresh(2)
    sampled_pc_np = np.asarray(fil.filter())
    print(sampled_pc_np.shape)

    # random selection
    choice_idx = np.random.choice(sampled_pc_np.shape[0], 16384, replace=False)
    sampled_pc_np = sampled_pc_np[choice_idx, :]
    print(sampled_pc_np.shape)
    
    ###### knn analysis
    knn_k=2
    knn = KNNBuilder(k=knn_k)

    start_t = timeit.default_timer()
    index = knn.build_nn_index(np.ascontiguousarray(sampled_pc_np[:, 0:3], dtype=np.float32))
    D, I = knn.search_nn(index, np.ascontiguousarray(sampled_pc_np[:, 0:3], dtype=np.float32), k=knn_k)
    stop_t = timeit.default_timer()
    print(stop_t - start_t)

    D = np.sqrt(np.fabs(D))
    D = D[:, knn_k-1:knn_k]
    print(D.shape)
    print('pc k=%d - mean %f, max %f, min %f' % (knn_k, np.mean(D), np.max(D), np.min(D)))
    ###### knn analysis
    
    break
    
    
#     fig = plt.figure()
#     ax = Axes3D(fig)
#     ax.scatter(pc_np[:,0].tolist(), pc_np[:,1].tolist(), pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
#     axisEqual3D(ax)

#     plt.ion()
#     plt.show()
#     break
    

In [4]:
# get surface normal
def Surface_normals(cloud):
    ne = cloud.make_NormalEstimation()
    tree = cloud.make_kdtree()
    ne.set_SearchMethod(tree)
#     ne.set_RadiusSearch(2)
    ne.set_KSearch(9)
    cloud_normals = ne.compute()
    return cloud_normals

# 1. voxel grid filer
# 2. apply outlier filter
# 3. calculate surface normal and curvature
def process_pc(pc_np):
#     print('origin')
#     print(pc_np.shape)
    
    # 1. voxel grid filer
    sampler = PCSampler(leaf_size=0.2, minimum_pc_num=20480)
    pc_np = sampler.sample_pc_wrapper(pc_np)
#     print('after voxel grid filter')
#     print(pc_np.shape)
    
    # 2. apply outlier filter
#     fil = pcl.PointCloud(pc_np).make_statistical_outlier_filter()
#     fil.set_mean_k(50)
#     fil.set_std_dev_mul_thresh(2)
#     pc_np = np.asarray(fil.filter())
#     print('after outlier filter')
#     print(pc_np.shape)
    
#     fig = plt.figure()
#     ax = Axes3D(fig)
#     ax.scatter(pc_np[:,0].tolist(), pc_np[:,1].tolist(), pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
#     axisEqual3D(ax)

#     plt.ion()
#     plt.show()
    
    # 3. calculate surface normal and curvature
    assert pc_np.shape[1] == 3
    cloud = pcl.PointCloud(pc_np)    
    sn = Surface_normals(cloud)
    sn_np = np.asarray(sn.to_array(), dtype=np.float32)  # Nx4, nx,ny,nz,curvature
    
    output_np = np.concatenate((pc_np, sn_np), axis=1)  # Nx7
    
    return output_np
    
    
def process_dataset(dataset, train_folder_path, output_folder):
    min_pc_num = 1000000
    for i, data in enumerate(dataset):
        # load bin
        bin_file_name = data['file']
        pc_np = load_bin(os.path.join(train_folder_path, bin_file_name))
        
        # process pc
        pc_np = process_pc(pc_np)
        
        # save pc as numpy array
        if not os.path.isdir(os.path.join(output_folder, bin_file_name[0:19])):
            os.makedirs(os.path.join(output_folder, bin_file_name[0:19]))
        npy_file_name = bin_file_name[0:-3] + 'npy'
        np.save(os.path.join(output_folder, npy_file_name), pc_np)
        
        if pc_np.shape[0] < min_pc_num:
            min_pc_num = pc_np.shape[0]
            
        if i % 100 == 0:
            print('%d - min_pc_num %d' % (i, min_pc_num))
    print('minimum point number: %d' % min_pc_num)

In [5]:
# train
train_folder_path = '/ssd/dataset/oxford/train'
train_txt_path = '/ssd/dataset/oxford/train_relative.txt'
dataset = load_train_txt(train_txt_path)
process_dataset(dataset, train_folder_path, '/ssd/dataset/oxford/train_np_nofilter')

# test
test_folder_path = '/ssd/dataset/oxford/test_models'
test_txt_path = '/ssd/dataset/oxford/test_models/groundtruths.txt'  # with rotation, translation gt, for each pair


0 - min_pc_num 37907
100 - min_pc_num 29706
200 - min_pc_num 25185
300 - min_pc_num 23140
400 - min_pc_num 21409
500 - min_pc_num 21361
600 - min_pc_num 21361
700 - min_pc_num 21361
800 - min_pc_num 20889
900 - min_pc_num 20889
1000 - min_pc_num 20889
1100 - min_pc_num 20889
1200 - min_pc_num 20889
1300 - min_pc_num 20889
1400 - min_pc_num 20889
1500 - min_pc_num 20889
1600 - min_pc_num 20889
1700 - min_pc_num 20698
1800 - min_pc_num 20698
1900 - min_pc_num 20698
2000 - min_pc_num 20698
2100 - min_pc_num 20698
2200 - min_pc_num 20698
2300 - min_pc_num 20698
2400 - min_pc_num 20698
2500 - min_pc_num 20698
2600 - min_pc_num 20698
2700 - min_pc_num 20698
2800 - min_pc_num 20698
2900 - min_pc_num 20698
3000 - min_pc_num 20698
3100 - min_pc_num 20630
3200 - min_pc_num 20630
3300 - min_pc_num 20630
3400 - min_pc_num 20630
3500 - min_pc_num 20630
3600 - min_pc_num 20630
3700 - min_pc_num 20630
3800 - min_pc_num 20630
3900 - min_pc_num 20630
4000 - min_pc_num 20630
4100 - min_pc_num 20630
4200

In [ ]:
pc_np = np.load('/ssd/dataset/oxford/train_np/2014-06-26-09-31-18/0.npy')  # x, y, z, nx, ny, nz, curvature
print(pc_np.dtype)
print(pc_np.shape)
print(pc_np)

# mean of three axis
print('max of three axis')
print(np.max(pc_np[:, 0:3], axis=0))
print('min of three axis')
print(np.min(pc_np[:, 0:3], axis=0))

# mean of norm
print('surface normal mean norm, should be 1')
print(np.mean(np.linalg.norm(pc_np[:, 3:6], axis=1)))

# statistics of curvature
print('curvature mean %f, min %f, max %f' % (np.mean(pc_np[:, 6]), np.min(pc_np[:, 6]), np.max(pc_np[:, 6])))


fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(pc_np[:,0].tolist(), pc_np[:,1].tolist(), pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
axisEqual3D(ax)

plt.ion()
plt.show()